In [1]:
# Read features extracted by CNNs
from utils.data_source import DataSource

In [2]:
# features_{n}/ where n represents the number of frames considered as input. 
!ls features/features_20

base_0            densenet121       mobilenetv2       vgg19
base_1            densenet169       nasnetlarge       xception
base_2            densenet201       nasnetmobile
base_3            efficientnetb0    resnet152v2
base_4            inceptionresnetv2 vgg16


In [3]:
# features, filenames and labels
!ls features/features_20/xception/

features-2048.npy filenames.npy     labels.npy


In [4]:
# get data from FOLD 0-4 (5 folds)
ds = DataSource(seq_length=20, features_length=2048, features_dir='features/features_20/', model='xception')

In [5]:
ds

In [6]:
# shows the number of videos by diagnostic class and partition
print(ds.count_videos(fold_number_str='0', data_type='train'))
print(ds.count_videos(fold_number_str='1', data_type='train'))
print(ds.count_videos(fold_number_str='2', data_type='train'))
print(ds.count_videos(fold_number_str='3', data_type='train'))
print(ds.count_videos(fold_number_str='4', data_type='train'))

[['cov' 'pne' 'reg']
 ['56' '40' '53']]
[['cov' 'pne' 'reg']
 ['56' '40' '53']]
[['cov' 'pne' 'reg']
 ['56' '40' '52']]
[['cov' 'pne' 'reg']
 ['55' '40' '53']]
[['cov' 'pne' 'reg']
 ['53' '40' '53']]


In [7]:
# shows the number of videos by diagnostic class and fold
print(ds.count_videos(fold_number_str='0', data_type='test'))
print(ds.count_videos(fold_number_str='1', data_type='test'))
print(ds.count_videos(fold_number_str='2', data_type='test'))
print(ds.count_videos(fold_number_str='3', data_type='test'))
print(ds.count_videos(fold_number_str='4', data_type='test'))

[['cov' 'pne' 'reg']
 ['13' '10' '13']]
[['cov' 'pne' 'reg']
 ['13' '10' '13']]
[['cov' 'pne' 'reg']
 ['13' '10' '14']]
[['cov' 'pne' 'reg']
 ['14' '10' '13']]
[['cov' 'pne' 'reg']
 ['16' '10' '13']]


In [8]:
# returns (filenames, labels)
x, y = ds.get_data_from_fold('0')

In [9]:
x[0], y[0]

('Pneu_northumbria_0409_set6_vid3.avi', 'pne')

In [10]:
# returns (features, labels) referring to fold 0.
x, y = ds.get_train_from_fold('0')

In [11]:
x.shape

(149, 20, 2048)

In [12]:
y.shape

(149, 3)

In [13]:
x[0].shape # 20 frames each frame containing 2048 features (Xception)

(20, 2048)

In [14]:
y[0] # Covid = [1,0,0]  Pneumonia = [0,1,0] Healthy = [0,0,1]

array([0, 1, 0])

In [19]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')


# model path
saved_model_path = 'best_model/xception-20/trial-16-fold-0-model.h5' # trained in fold 0

# load model
model = tf.keras.models.load_model(saved_model_path)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               5244928   
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 6,822,915
Trainable params: 6,822,915
Non-trainable params: 0
_________________________________________________________________


In [21]:
from sklearn.metrics import classification_report
import numpy as np

accs = []; cprecs = []; crecs=[]; cf1s=[]; pprecs = []; precs=[]; pf1s=[]; hprecs = []; hrecs=[]; hf1s=[];

    
# tests the model on each of the five partitions
for i in range(5):
    # Load features and labels
    x_train, y_train = ds.get_train_from_fold(str(i))
    X_test, Y_test = ds.get_test_from_fold(str(i))
    
    saved_model_path = f"./best_model/xception-20/trial-16-fold-{i}-model.h5"
    model = tf.keras.models.load_model(saved_model_path)
    
    predictions = model.predict(X_test)

    score = classification_report(
        y_true=np.argmax(Y_test, axis=1), 
        y_pred=np.argmax(predictions, axis=1), 
        labels=[0,1,2], 
        target_names=['covid', 'pneumonia', 'healthy'], 
        zero_division=0,
        output_dict=True
    )
    
    accs.append(score['accuracy'])
    cprecs.append(score['covid']['precision'])
    crecs.append(score['covid']['recall'])
    cf1s.append(score['covid']['f1-score']) 
    pprecs.append(score['pneumonia']['precision'])
    precs.append(score['pneumonia']['recall'])
    pf1s.append(score['pneumonia']['f1-score'])
    hprecs.append(score['healthy']['precision'])
    hrecs.append(score['healthy']['recall'])
    hf1s.append(score['healthy']['f1-score'])    

In [22]:
print("Model: Xception (mean, std)")
print("Accuracy: ", (round(np.mean(accs), 2), round(np.std(accs), 2)))
print("COVID-19 Precision: ", (round(np.mean(cprecs), 2), round(np.std(cprecs), 2)))
print("Pneumonia Precision: ", (round(np.mean(pprecs), 2), round(np.std(pprecs), 2)))
print("Healthy Precision: ", (round(np.mean(hprecs), 2), round(np.std(hprecs), 2)))
print("COVID-19 Recall: ", (round(np.mean(crecs), 2), round(np.std(crecs), 2)))
print("Pneumonia Recall: ", (round(np.mean(precs), 2), round(np.std(precs), 2)))
print("Healthy Recall: ", (round(np.mean(hrecs), 2), round(np.std(hrecs), 2)))
print("COVID-19 F1: ", (round(np.mean(cf1s), 2), round(np.std(cf1s), 2)))
print("Pneumonia F1: ", (round(np.mean(pf1s), 2), round(np.std(pf1s), 2)))
print("Healthy F1: ", (round(np.mean(hf1s), 2), round(np.std(hf1s), 2)))

Model: Xception (mean, std)
Accuracy:  (0.93, 0.13)
COVID-19 Precision:  (0.94, 0.12)
Pneumonia Precision:  (0.92, 0.17)
Healthy Precision:  (0.95, 0.1)
COVID-19 Recall:  (0.97, 0.06)
Pneumonia Recall:  (0.94, 0.12)
Healthy Recall:  (0.89, 0.22)
COVID-19 F1:  (0.95, 0.1)
Pneumonia F1:  (0.93, 0.15)
Healthy F1:  (0.91, 0.17)
